In [ ]:
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

## Load in Network

In [ ]:
cubes = 'control_network_metrics/registration_quality/cubes.lis'
sensors = generate_sensors(cubes, directory='data_lak/tmp/', clean=True)

network_path = 'control_network_metrics/registration_quality/measure_error_50px_1pts_5mean_2std.net'
network = io_controlnetwork.from_isis(network_path)
network = compute_apriori_ground_points(network, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

## Determine Solve Parameters

In [ ]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity

## Functions

In [ ]:
def compute_sigma(V, W_parameters, W_observations):
    """
    Computes the resulting standard deviation of the residuals for the current state of the bundle network.
    
    Parameters
    ----------
    V  :  np.array
          The control network dataframe with updated ground points
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : float64
         Standard deviation of the residuals
    
    """
    num_parameters = W_parameters.shape[0]
    num_observations = W_observations.shape[0]
    dof = num_observations - num_parameters
    VTPV = (V.dot(W_observations).dot(V))
    sigma0 = np.sqrt(VTPV/dof)
    return sigma0

def bundle_iteration(J, V, W_parameters, W_observations):
    """
    Parameters
    ----------
    J  :  ndarray
          The control network as a dataframe generated by plio.
    V  :  np.array
          The control network dataframe with updated ground points
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: measure weights)
    
    Returns
    -------
    N  :  
    """
    
    N = J.T.dot(W_observations).dot(J) + W_parameters
    C = J.T.dot(W_observations).dot(V)
    dX = np.linalg.inv(N).dot(C)
    return N, dX

# For data snooping we need to calculate updated residuals
def compute_normalized_residual(J, V, N, W_parameters, W_observations):
    """
    Computes the normalized residual statistic for the data snooping method. Method derived from 
    Forstner 1985 "The Reliability of Block Triangulation"
    
    Parameters
    ----------
    V  :  np.array
          The control network dataframe with updated ground points
    N  :  
        
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : np.array
         Normalized residual statistic for the data snooping
    
    """
    sigma0 = compute_sigma(V, W_parameters, W_observations)
    Qxx = np.linalg.inv(N)
    Qvv = np.linalg.inv(W_observations) - J.dot(Qxx).dot(J.T)
    qvv = np.diagonal(Qvv)
    sigma_vi = sigma0*np.sqrt(qvv)
    wi = -V/sigma_vi
    
    return wi

## Data Snooping Function

In [ ]:
k = 3.29 #critical values from Forstner
# k = 4.1 #cricital value from Baarda

In [ ]:
def data_snooping(network, sensors, parameters, k):
    """
    Parameters
    ----------
    network  :  DataFrame
                The control network as a dataframe generated by plio
    sensors  :  dict
                A dictionary that maps ISIS serial numbers to CSM sensors
    parameters  : list
                 The list of  CsmParameter to compute the partials W.R.T.
    k  :  float64
          Critical value used for rejection criteria; defaults to Forstner's 3.29 
          (or Baarda's 4.1??)
    
    Returns
    -------
      :  list
      Indices of the network DataFrame that were rejected during data snooping
    """
    net = network 
    net['mask'] = "True"

    rejected_indices = []
    awi = np.array([5, 5, 5, 5]) #initialize larger than k so you get into first iteration
    while (awi > k).any():
        print(len(net[net['mask']=='True']))

        # weight matrices
        coefficient_columns = compute_coefficient_columns(network, sensors, parameters)
        num_parameters = max(col_range[1] for col_range in coefficient_columns.values())
        W_parameters = compute_parameter_weights(network, sensors, parameters, coefficient_columns)
        num_observations = 2 * len(net[net['mask']=="True"])
        W_observations = np.eye(num_observations)

        # bundle iteration (and set up)
        V = compute_residuals(net[net['mask']=="True"], sensors)
        J = compute_jacobian(net[net['mask']=="True"], sensors, parameters, coefficient_columns)
        sigma0 = compute_sigma(V, W_parameters, W_observations)
        N, dX = bundle_iteration(J, V, W_parameters, W_observations)

        # calculate test statistic
        wi = compute_normalized_residual(J, V, N, W_parameters, W_observations)
        awi = abs(wi)

        #find maximum
        imax = np.argmax(awi)
        print(f'max wi = {awi[imax]}') # display

        if awi[imax] <= k:
            print('Data Snooping Outlier Rejection Complete')
            break

        else:
            reject = net.index[net['mask']=="True"][imax]
            net.loc[net.index == reject, ['mask']] = "False"
            rejected_indices.append(reject)
            
    return rejected_indices


In [ ]:
rejected_indices = data_snooping(network, sensors, parameters, k)

In [ ]:
# plt.figure()
# plt.boxplot(wi)
plt.figure()
plt.hist(wi[abs(wi) < k], bins=np.linspace(-4,5,50));
plt.hist(wi[abs(wi) > k], bins=np.linspace(-4,5,50));